In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tomllib
import shapely

In [ ]:
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.nddata import CCDData

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
import read_iso
from ana_utils import is_in_poly


In [ ]:
obj = "yasone1"
julen = True

In [ ]:
r_h = 25 / 60

In [ ]:
r_inner = 40  / 60
r_ring = 2*r_inner

In [ ]:
if obj == "yasone1":
    ra0, dec0 = 265.52019, 13.17146
elif obj == "yasone2":
    ra0, dec0 = 262.34921, 6.42302
elif obj == "yasone3":
    ra0, dec0 = 292.96258, -26.44994

In [ ]:
coord0 = SkyCoord(ra0, dec0, unit=u.degree)
r_0 = 25 / 0.254 

In [ ]:
isoname = "../MIST/MIST_v1.2_vvcrit0.4_PanSTARRS/MIST_v1.2_feh_m1.50_afe_p0.0_vvcrit0.4_PanSTARRS.iso.cmd"
iso_age = 12e9

# Utilities, etc

In [ ]:
def tangent_axis():
    plt.ylabel(r"$\eta/\textrm{arcmin}$")
    plt.xlabel(r"$\xi/\textrm{arcmin}$")
    plt.xlim(3, -3)
    plt.ylim(-3, 3)
    plt.xticks(np.arange(-3, 4))
    plt.yticks(np.arange(-3, 4))
    plt.gca().set_aspect(1)

In [ ]:
def ri_axis():
    plt.xlabel(r"$r-i$ (mag)")
    plt.ylabel(r"$r$ (mag)")

    plt.xlim(-0.5, 1.5)
    plt.ylim(27, 17)
    plt.gca().set_box_aspect(1)


def gr_axis():
    plt.xlabel(r"$g-r$ (mag)")
    plt.ylabel(r"$g$ (mag)")
    plt.xlim(-0.5, 1.5)
    plt.ylim(27, 17)
    plt.gca().set_box_aspect(1)


In [ ]:
def to_coords(cat):
    return SkyCoord(cat["ALPHA_J2000"], cat["DELTA_J2000"], unit=u.degree)


In [ ]:
def select_centre(cat, coord0, radius):
    filt = coord0.separation(to_coords(cat)) < radius
    return cat[filt]

In [ ]:
def get_upper_radius(radius, radius_bkg):
    return np.sqrt(radius**2 + radius_bkg**2)


In [ ]:
def select_annuli(cat, coord0, radius, radius_bkg):
    radius_upper = get_upper_radius(radius, radius_bkg)
    # print(radius**2, radius_upper**2 - radius_bkg**2)
    seps = coord0.separation(to_coords(cat))

    filt = seps >= radius_bkg
    filt &= seps < radius_upper
    return cat[filt]

In [ ]:
def plot_annuli(radius, radius_bkg):
    circ = plt.Circle((0, 0), radius, color="black", lw=1, fill=False)
    circ2 = plt.Circle((0, 0), radius_bkg, color="black", lw=1, fill=False)
    circ3 = plt.Circle((0, 0), get_upper_radius(radius, radius_bkg), color="black", lw=1, fill=False)
    ax = plt.gca()
        
    ax.add_patch(circ)
    ax.add_patch(circ2)
    ax.add_patch(circ3)

In [ ]:
def to_tangent(coord, coord0):
    alpha = coord.ra.to("rad").value
    delta = coord.dec.to("rad").value
    alpha0 = coord0.ra.to("rad").value
    delta0 = coord0.dec.to("rad").value


    denom = np.sin(delta) * np.sin(delta0) + np.cos(delta) * np.cos(delta0) * np.cos(alpha - alpha0)

    xi_num = np.cos(delta) * np.sin(alpha - alpha0)
    eta_num = np.sin(delta) * np.cos(delta0) - np.cos(delta) * np.sin(delta0) * np.cos(alpha-alpha0) 

    xi = np.rad2deg(xi_num/denom)
    eta = np.rad2deg(eta_num / denom)
    return xi, eta

In [ ]:
def read_catalogue_julen(objname):
    cat = Table.read(f"{objname}_julen.cat")

    return cat

# Data loading

In [ ]:
cat = read_catalogue_julen(obj)

In [ ]:
xi, eta = to_tangent(to_coords(cat), coord0)
cat["xi"] = xi * 60 * u.arcmin
cat["eta"] = eta * 60 * u.arcmin

In [ ]:
iso_ps = read_iso.ISOCMD(isoname)[np.log10(iso_age)]

In [ ]:
cat_all = read_catalogue_julen(obj)
cat_all = cat_all[cat_all["ellipticity_r"] < 1]

xi, eta = to_tangent(to_coords(cat_all), coord0)
cat_all["xi"] = xi * 60 * u.arcmin
cat_all["eta"] = eta * 60 * u.arcmin

In [ ]:
cat_gr = cat_all[cat_all["distance_gr"] * 3600 < 1]
cat_ri = cat_all[cat_all["distance_ri"] * 3600 < 3]

In [ ]:
plt.scatter(cat_ri["xi"], cat_ri["eta"])

In [ ]:
def err_model(x, a, b, c):
    return np.log10(np.maximum(1e-12, 10**(x * a + b) + c))

In [ ]:
x = cat_gr["mag_g_ps"]
y = cat_gr["mag_gr_ps_err"]
filt = np.isfinite(y) & (y > 0)

gpopt, covt = curve_fit(err_model, x[filt], np.log10(y[filt]))
gpopt, np.sqrt(np.diag(covt))

In [ ]:
x = cat_ri["mag_r_ps"]
y = cat_ri["mag_ri_ps_err"]
filt = np.isfinite(y) & (y > 0)
rpopt, covt = curve_fit(err_model, x[filt], np.log10(y[filt]))
rpopt, np.sqrt(np.diag(covt))

In [ ]:
iso_width = 0.05

In [ ]:
def gmag_err(gmag):
    return 10**err_model(gmag, *gpopt)

def rmag_err(mag):
    return 10**err_model(mag, *rpopt)

In [ ]:
fig, axs = plt.subplots(1, 2)

plt.sca(axs[0])
plt.scatter(cat_all["mag_g_ps"], cat_all["mag_gr_ps_err"], s=0.3, lw=0, alpha=0.1, color="k")

x = np.linspace(15, 28)
y = gmag_err(x)
plt.plot(x, y)
plt.yscale("log")
# plt.xlim(0, 1e3)
plt.ylim(1e-3, 2)

plt.ylabel("err")
plt.xlabel("g")

plt.sca(axs[1])
plt.scatter(cat_all["mag_r_ps"], cat_all["mag_ri_ps_err"], s=0.3, lw=0, alpha=0.1, color="k")

x = np.linspace(15, 28)
y = rmag_err(x)
plt.plot(x, y)
plt.yscale("log")
# plt.xlim(0, 1e3)
plt.ylim(1e-3, 2)
plt.xlabel("r")




In [ ]:
A_ri = 0.1
A_gr = 0.3

In [ ]:
def make_polygon(iso, mag_err, dm, A=A_ri, iso_width=0.05, yfilt="SDSS_r", xfilt="SDSS_i"):
    filt = iso["phase"] < 3

    x = iso[yfilt][filt].data - iso[xfilt][filt].data + A
    y = iso[yfilt][filt].data + dm

    x_min = x - gmag_err(y) - iso_width
    x_max = x + gmag_err(y) + iso_width

    return np.hstack([x_min, x_max[::-1], x_min[0]]), np.hstack([y, y[::-1], y[0]])


In [ ]:
def plot_iso_ps(dm=dm, A=A_gr, phase_max=3):
    filt = iso_ps["phase"] < phase_max
    plt.plot(iso_ps["PS_g"][filt] - iso_ps["PS_r"][filt] + A, iso_ps["PS_g"][filt] + dm, color="k")

    
    x_poly, y_poly = make_polygon(iso_ps, gmag_err, iso_width=0, dm=dm, A=A, yfilt="PS_g", xfilt="PS_r")
    plt.plot(x_poly, y_poly, color="k")

    
def plot_iso_ps_ri(dm=dm, A=A_ri, phase_max=3):
    filt = iso_ps["phase"] < phase_max
    plt.plot(iso_ps["PS_r"][filt] - iso_ps["PS_i"][filt] + A, iso_ps["PS_r"][filt] + dm, color="k")
    
    x_poly, y_poly = make_polygon(iso_ps, rmag_err, iso_width=0, dm=dm, A=A, yfilt="PS_r", xfilt="PS_i")
    plt.plot(x_poly, y_poly, color="k")


In [ ]:
plt.rcParams["text.usetex"] = True
plt.rcParams["figure.dpi"] = 150

In [ ]:
def plot_cmd(sample, A, dm, b="g", r="r", s_in=3):
    bmag = sample[f"mag_{b}_ps"]
    rmag = sample[f"mag_{r}_ps"]

    plt.scatter(bmag - rmag, bmag, alpha=0.5, color="grey", s=1)

    if b == "g":
        sample_cmd = select_gr(sample)
    else:
        sample_cmd = select_ri(sample)
        
    bmag = sample_cmd[f"mag_{b}_ps"]
    rmag = sample_cmd[f"mag_{r}_ps"]

    plt.scatter(bmag - rmag, bmag, alpha=0.5, color=b, s=s_in)

    if b == "g":
        plot_iso_ps(A=A, dm=dm)
        plt.ylim(28)
        plt.xlim(-1, 2)
    else:
        plot_iso_ps_ri(A=A, dm=dm)

In [ ]:
A_gr = -0.4
A_ri = -0.1
dm = 16.5

In [ ]:
x_poly_ri, y_poly_ri = make_polygon(iso_ps, rmag_err, dm, A=A_ri, xfilt="PS_i", yfilt="PS_r", iso_width=0)

In [ ]:
x_poly_gr, y_poly_gr = make_polygon(iso_ps, gmag_err, dm, A=A_gr, xfilt="PS_r", yfilt="PS_g", iso_width=0)

In [ ]:
def select_gr(cat_gr):

    cmd_filt_gr = is_in_poly(cat_gr["mag_g_ps"].data - cat_gr["mag_r_ps"].data, cat_gr["mag_g_ps"].data, x_poly_gr, y_poly_gr)

    return cat_gr[cmd_filt_gr]

In [ ]:
def select_ri(cat_ri):

    cmd_filt_ri = is_in_poly(cat_ri["mag_r_ps"].data - cat_ri["mag_i_ps"].data, cat_ri["mag_r_ps"].data, x_poly_ri, y_poly_ri)

    return cat_ri[cmd_filt_ri]

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10, 5))


############### g vs r
sample_bkg = select_annuli(cat_gr, coord0, r_inner * u.arcmin, r_ring*u.arcmin)
sample_cen = select_centre(cat_gr, coord0, r_inner * u.arcmin )
sample_cen_cmd = select_gr(sample_cen)
sample_bkg_cmd = select_gr(sample_bkg)
sample_cmd = select_gr(cat_gr)

plt.sca(axs[0][0])


plt.scatter(cat_gr["xi"], cat_gr["eta"], s=1, color="grey", lw=0)
plt.scatter(sample_cmd["xi"], sample_cmd["eta"], s=3,  lw=0.0, ec="k")
plt.scatter(sample_cen["xi"], sample_cen["eta"], s=15, lw=0.3, ec="k")
tangent_axis()
plot_annuli(r_inner, r_ring)

plt.xlim(-3, 3)


plt.sca(axs[0][1])
gr_axis()
plot_cmd(sample_cen, A=A_gr, dm=dm, s_in=10)



plt.sca(axs[0][2])
gr_axis()
plot_cmd(sample_bkg, A=A_gr, dm=dm)



################# r vs i

sample_bkg = select_annuli(cat_ri, coord0, r_inner * u.arcmin, r_ring*u.arcmin)
sample_cen = select_centre(cat_ri, coord0, r_inner * u.arcmin )
sample_cen_cmd = select_ri(sample_cen)
sample_cmd = select_ri(cat_ri)
sample_bkg_cmd = select_ri(sample_bkg)


plt.sca(axs[1][0])
plt.scatter(cat_ri["xi"], cat_ri["eta"], s=1, color="grey", lw=0)
plt.scatter(sample_cen_cmd["xi"], sample_cen_cmd["eta"], s=10, lw=0.3, ec="k")
plt.scatter(sample_cmd["xi"], sample_cmd["eta"], s=3,  lw=0.0, ec="k")
tangent_axis()
plot_annuli(r_inner, r_ring)
plt.xlim(-3, 3)




plt.sca(axs[1][1])
ri_axis()
plot_cmd(sample_cen, b="r", r="i", A=A_ri, dm=dm, s_in=10)


plt.sca(axs[1][2])
ri_axis()
plot_cmd(sample_bkg, b="r", r="i", A=A_ri, dm=dm)


plt.ylim(27)





axs[1][1].set_box_aspect(0.9)
axs[1][2].set_box_aspect(0.9)
axs[0][1].set_box_aspect(0.9)
axs[0][2].set_box_aspect(0.9)

plt.tight_layout()

In [ ]:
plt.hist(cat_all["mag_r_ps"], 50)